In [1]:
!pip install tk
!pip install PyDictionary
!pip install pyttsx3
!pip install ipywidgets
!pip install gtts

  Obtaining dependency information for tk from https://files.pythonhosted.org/packages/1e/0b/029cbdb868bb555fed99bf6540fff072d500b3f895873709f25084e85e33/tk-0.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for PyDictionary from https://files.pythonhosted.org/packages/d5/0b/575893d32fd0484f3d05a0d754fd35226fee7257b433576484cd7455e429/PyDictionary-2.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for bs4 from https://files.pythonhosted.org/packages/51/bb/bf7aab772a159614954d84aa832c129624ba6c32faa559dfb200a534e50b/bs4-0.0.2-py2.py3-none-any.whl.metadata
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [30 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 14, in <module>
    File "C:\Users\Admin\anaconda3\Lib\site-packages\setuptools\__init__.py", line 270, in <module>
      monkey.patch_all()
    File "C:\Users\Admin\anaconda3\Lib\site-packages\setuptools\monkey.py", line 97, in patch_all
      patch_for_msvc_specialized_compiler()
    File "C:\Users\Admin\anaconda3\Lib\site-packages\setuptools\monkey.py", line 157, in patch_for_msvc_specialized_compiler
      patch_func(*msvc14('_get_vc_env'))
                  ^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\Admin\anaconda3\Lib\site-packages\setuptools\monkey.py", line 147, in patch_params
      mod = import_module(mod_name)
            ^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\Admin\anaconda3\Lib\importlib\__init__.py", line 126, in import_

  Obtaining dependency information for pyttsx3 from https://files.pythonhosted.org/packages/e4/ed/beb8596ee4721ee44000aeab29e1ad8c6b7634d1e72d536fa1e9e9644d88/pyttsx3-2.91-py3-none-any.whl.metadata
  Obtaining dependency information for comtypes from https://files.pythonhosted.org/packages/91/85/3885c830af528337619f8bc8d0d3e56576077cafbb1b51977ffc7e17888c/comtypes-1.4.6-py3-none-any.whl.metadata
  Obtaining dependency information for pypiwin32 from https://files.pythonhosted.org/packages/d0/1b/2f292bbd742e369a100c91faa0483172cd91a1a422a6692055ac920946c5/pypiwin32-223-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/222.1 kB ? eta -:--:--
   ------- -------------------------------- 41.0/222.1 kB ? eta -:--:--
   ----------- ---------------------------- 61.4/222.1 kB 3.2 MB/s eta 0:00:01
   ----------------------------- ---------- 163.8/222.1 kB 1.6 MB/s eta 0:00:01
   ----------------------------- ---------- 163.8/222.1 kB 1.6 MB/s eta 0:00:01
   ---------------

In [2]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...


True

In [4]:
import nltk
import os
import time
import re
from nltk.corpus import wordnet
from gtts import gTTS
from IPython.display import display, Audio, Markdown
import ipywidgets as widgets

# Download required NLTK data
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

# Queue to manage audio playback
audio_queue = []

def remove_html_tags(text):
    """Remove HTML tags from text."""
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def speak(text, remove_audio=True):
    """Generate and queue audio for text-to-speech."""
    plain_text = remove_html_tags(text)
    tts = gTTS(text=plain_text, lang='en')
    audio_file = "output.mp3"
    tts.save(audio_file)
    # Estimate audio duration (words per second)
    audio_duration = len(plain_text.split()) / 2.5 
    audio_queue.append((audio_file, audio_duration))
    if len(audio_queue) == 1:
        play_next_audio(remove_audio)

def play_next_audio(remove_audio=True):
    """Play the next audio in the queue."""
    if audio_queue:
        audio_file, audio_duration = audio_queue[0]
        display(Audio(audio_file, autoplay=True))
        time.sleep(audio_duration + 2) # Reduced buffer time
        if remove_audio:
            try:
                os.remove(audio_file)
            except OSError:
                pass # Ignore if file doesn't exist or can't be removed
        audio_queue.pop(0) # Remove the played audio file from the queue
        if audio_queue:
            play_next_audio(remove_audio) # Play the next audio if queue is not empty

def get_meaning(change=None):
    """Get and display the meaning of a word."""
    word = text.value.strip().lower() # Convert input to lowercase
    if not word:
        return
    try:
        synsets = wordnet.synsets(word)
        if not synsets:
            raise ValueError(f"No definitions found for '{word}'")
        # Clear previous output
        output.clear_output(wait=True)
        with output:
            display(Markdown(f"*Word:* {text.value.strip()}"))
            for synset in synsets:
                pos_name = {'n': 'Noun', 'v': 'Verb', 'a': 'Adjective', 'r': 'Adverb'}.get(synset.pos(), synset.pos().capitalize())
                meaning_text = f"{pos_name}: {synset.definition().strip()}"
                display(Markdown(meaning_text))
                speak(f"{pos_name}. {synset.definition().strip()}", remove_audio=False)
        text.value = "" # Clear input after processing
    except ValueError as ve:
        output.clear_output(wait=True)
        with output:
            display(Markdown(f"*Error:* {str(ve)}"))
        speak(f"No definitions found for '{word}'")
    except Exception as e:
        output.clear_output(wait=True)
        with output:
            display(Markdown(f"*Error:* {str(e)}"))
        speak(f"An error occurred: {str(e)}")

# Create widgets
text = widgets.Text(description='Word:', placeholder='Type a word')
button = widgets.Button(description='Find Meaning', button_style='info', icon='search')
output = widgets.Output()

# Connect widgets
text.on_submit(lambda _: get_meaning())
button.on_click(get_meaning)

# Display widgets
display(widgets.VBox([
    widgets.HTML('<h2>SYNONYMS:</h2>'),
    widgets.HBox([text, button]),
    output
]))

# Initial prompt
speak("Which word would you like to find the meaning of?")

C:\Users\Admin\AppData\Local\Temp\ipykernel_20488\3548051095.py:85: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  text.on_submit(lambda _: get_meaning())
